In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import json
import yaml
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
import requests

In [3]:
def get_wikipedia(lang, name):
    # https://stackoverflow.com/a/28401782
    response = requests.get(
        f"https://{lang}.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&titles={name}"
    )
    pages = response.json()["query"]["pages"]
    if "-1" in pages.keys():
        return None
    else:
        for _, v in pages.items():
            return v

In [4]:
df = pd.read_csv("./对照表 3.csv")

In [5]:
df = df.fillna("")

In [6]:
df.head()

,原文,中译,属性,人物与另一词条相等情况,人物所属另一词条（团体）,作品所属另一词条（作者）,作品所属另一词条（专辑）（单曲或章节等情况下）,作品年份,mimida,翻译来源及注释（mimida）,Tian Tang,翻译来源及注释（Tian Tang）,redhousepainter,翻译来源及注释（redhousepainter）,Lil Bag,翻译来源及注释（Lil Bag）
0,World Class Wreckin' Cru,世界级破坏团伙,乐人,,,,,,世界级破坏团伙,,,,,,,
1,《Juice》,果汁,歌,,,World Class Wreckin' Cru,,1985,果汁,,,,,,,
2,Alonzo Williams,阿隆佐·威廉姆斯,乐人,,World Class Wreckin' Cru,,,,阿隆佐·威廉姆斯,,,,,,,
3,DJ Yella,DJ耶拉,乐人,,World Class Wreckin' Cru,,,,DJ耶拉,,,,,,,
4,Ice T,冰T,乐人,,,,,,冰T,,,,,,,


In [7]:
data = []
for _, entry in df.iterrows():
    # handle empty row
    if not entry["原文"]:
        continue
    datum = {}
    datum["id"] = entry["原文"].replace("《", "").replace("》", "").lower().split("/")[0]
    datum["name"] = entry["原文"].replace("《", "").replace("》", "").split("/")[0]
    datum["consensus_translation"] = entry["中译"]
    datum["category"] = entry["属性"]
    datum["alias"] = entry["人物与另一词条相等情况"]
    datum["group"] = entry["人物所属另一词条（团体）"]
    datum["author"] = entry["作品所属另一词条（作者）"]
    datum["album"] = entry["作品所属另一词条（专辑）（单曲或章节等情况下）"]
    datum["date"] = entry["作品年份"]
    datum["language"] = "en"
    # the hacky way kind of fit for our datasets...
    try:
        datum["language"] = (
            "jp" if detect(datum["name"])[:2] in ["zh", "jp", "ko"] else "en"
        )
    except LangDetectException as e:
        print(f'failed to detect {entry["原文"]} language: {e}')
    datum["wikipedia"] = get_wikipedia(datum["language"], datum["name"])

    opinions = []
    for user in ["mimida", "Tian Tang", "redhousepainter", "Lil Bag"]:
        opinion = {}
        if entry[user]:
            opinion["user_name"] = user
            opinion["github_handle"] = (
                "idmimida" if user == "mimida" else user.replace(" ", "-")
            )
            opinion["translation"] = entry[user]
            opinion["details"] = entry[f"翻译来源及注释（{user}）"]
            opinions.append(opinion)

    datum["opinions"] = opinions

    data.append(datum)

failed to detect 《2001》 language: No features in text.
failed to detect 《4, 3, 2, 1》 language: No features in text.


In [8]:
def save_to_json(result):
    with open(f"../entries/{result['id']}.json", "w+") as fp:
        json.dump(result, fp, ensure_ascii=False)
    return

In [9]:
def save_to_yaml(result):
    with open(f"../entries/{result['id']}.yaml", "w+") as fp:
        yaml.dump(result, fp, allow_unicode=True)
    return

In [10]:
for d in data:
    save_to_json(d)
    # save_to_yaml(d)